
    
    
## <H2 align="center"> <font color='white'> **MAF1731B**: Microstructure and Trading Systems </H2>

### <H3 align="center"> <font color='white'> **Laboratory 4**: Market Microstructure </H3>

 <H5 align="center"> <em> Professor: Villalobos Ramos Omar Antonio <em>

 <p align="center"> <img style=" margin: 30px 15px 15px 15px;" src="https://pngimage.net/wp-content/uploads/2018/06/logo-iteso-png-5.png" width="350" height="150" /> 

 <strong> <H5 align="left"> Student:
Esteban Márquez Delgado <strong>

 *Registry*:
if700637: https://github.com/EstebanMqz/MyST_LAB_4

 <center> <font color= #555555> <font size = 4> November 2022 </a> <font color= #555555> <font size = 4> </a></font>

<hr style="border:0.02in solid gray"> </hr>

<center> <font color= #555555> <font size = 6> Laboratory 4 </font> <br> <br> <font color= #555555> <font size = 5> Market Microstructure  </font>

<hr style="border:0.02in solid gray"> </hr>

### <font color= #555555> <font size = 4> Abstract </font>

This document was prepared as delivery for Laboratory 4 of Microstructure and Trading Systems - MAF1731B class during the Autumn 2022 course at ITESO for the Bachelor of Financial Engineering. In order to analyze the market microstructure in criptocurrencies markets containing the following 3 sections:

1. **Consume CCXT data**: Download Criptocurrencies Order Books from exchanges (kraken, ftx, currencycom, coinmate, etc).

2. **Microstructure Visualization**: Build Time Series using the Order Books obtained in the previous point from the following elements of the Market Microstructure for each asset and exchange (Levels, Bid Volume, Ask Volume, Total Volume, Mid Price, VWAP.)

3. **Microstructure Modeling**: Estimation of Roll Effective Spread using the lagged series at t-5.



<font color= lightblue> <font size = 4> 0. Libraries and Dependencies: </font>

<font color= gray>

**Note**: To run this notebook it is necessary to have the following libraries contained in the [requirements.txt](https://github.com/EstebanMqz/MyST_LAB_4/blob/main/requirements.txt) script of this project.

To install the libraries in the script run the following cell:

In [1]:
!pip install -r requirements.txt

ERROR: Invalid requirement: '"""' (from line 1 of requirements.txt)


If you prefer, manual installation can be done with the individual installation of the following libraries:

<font color= lightblue> <font size = 1> 
+ pandas >= 1.3.4
+ numpy >= 1.19.1
+ jupyter >= 1.0.0
+ chart_studio >= 1.1
+ plotly >= 4.14
+ logging >= 3.11
+ datetime >= 4.7
+ pandas_datareader >= 0.10.0
+ tk >= 0.10
+ random >= 3.11
+ plotly >= 5.60
+ yahoofinancials >= 1.60
+ scipy >= 1.7.3
+ regex >= 2022.3.15
+ matplotlib >= 3.5.1
+ fire >= 0.4.0
+ ccxt >= 2.1.33
+ time >= 3.11 


### <font color= lightblue> <font size = 4> 0.1 Introduction: </font>
In this laboratory we will use information from the Cryptocurrency Order Book of various international exchanges, with the primary objective of visualizing and modeling the microstructure of the market through the behavior of the Order Book and its characteristics such as; Matching Engine, Levels, Ask-Bid Volumes, etc.
<font color= gray>




##### *General help:*
<font color= lightgreen> **Note**: Progress checks at the end of each section are highlighted. <font color= white>

In order, to retrieve specific help from the scripts tree in the project execute the help function on the script or in specific modules from the following:
+ [data.py](https://github.com/EstebanMqz/MyST_LAB_4/blob/main/data.py)
+ [functions.py](https://github.com/EstebanMqz/MyST_LAB_4/blob/main/functions.py)
+ [visualizations.py](https://github.com/EstebanMqz/MyST_LAB_4/blob/main/visualizations.py)

In [2]:
## Libraries
import pandas as pd # Analysis and data management
import pandas_datareader as pdr
import pandas_datareader.data as web #Data extraction from the web
import numpy as np
from datetime import datetime, timedelta, date
from tkinter.ttk import Style
from tkinter import Y
import random as rand
import plotly.graph_objects as go #Plotly
import plotly.express as px
import warnings
from yahoofinancials import YahooFinancials # Yfinance 
import yfinance as yf
import ccxt #Criptocurrencies
import logging as log #Exchanges 
from scipy.optimize import minimize # Optimization 
from collections import Counter #Counter
import time
import re
import io 
import glob
import os
#.Py archives in repo
import functions as fn 
import visualizations as vs
import data as dt
from os import path
import fire
%matplotlib inline

<font color= orange> <font size = 5> 1. Consume CCXT data: </font>

In order to know the exchanges avaliable in CCTX module in Python for Criptocurrencies data download, the following getattr function *find_exchanges* is executed:

In [3]:
help(fn.find_exchanges)

Help on function find_exchanges in module functions:

find_exchanges(features=None, is_authenticated=False)
    Function that returns avaliable cryptocurrencies exchanges in Python for CCTX module.
    
        Parameters:
        ----------
        features: None 
        is_authenticated: None 
    
        Returns:
        -------
        exchange_names: Array of authenticated exchanges avaliable in CCTX.



From all the 120 exchanges avaliable we pick **binance**, **ftx** and **bytetrade** because of their fast fetching.

In [4]:
#Available exchanges in CCTX
fn.find_exchanges()[:] #len 120

['aax',
 'alpaca',
 'ascendex',
 'bequant',
 'bibox',
 'bigone',
 'binance',
 'binancecoinm',
 'binanceus',
 'binanceusdm',
 'bit2c',
 'bitbank',
 'bitbay',
 'bitbns',
 'bitcoincom',
 'bitfinex',
 'bitfinex2',
 'bitflyer',
 'bitforex',
 'bitget',
 'bithumb',
 'bitmart',
 'bitmex',
 'bitopro',
 'bitpanda',
 'bitrue',
 'bitso',
 'bitstamp',
 'bitstamp1',
 'bittrex',
 'bitvavo',
 'bkex',
 'bl3p',
 'blockchaincom',
 'btcalpha',
 'btcbox',
 'btcex',
 'btcmarkets',
 'btctradeua',
 'btcturk',
 'buda',
 'bw',
 'bybit',
 'bytetrade',
 'cex',
 'coinbase',
 'coinbaseprime',
 'coinbasepro',
 'coincheck',
 'coinex',
 'coinfalcon',
 'coinmate',
 'coinone',
 'coinspot',
 'crex24',
 'cryptocom',
 'currencycom',
 'delta',
 'deribit',
 'digifinex',
 'exmo',
 'flowbtc',
 'fmfwio',
 'ftx',
 'ftxus',
 'gate',
 'gateio',
 'gemini',
 'hitbtc',
 'hitbtc3',
 'hollaex',
 'huobi',
 'huobijp',
 'huobipro',
 'idex',
 'independentreserve',
 'indodax',
 'itbit',
 'kraken',
 'kucoin',
 'kucoinfutures',
 'kuna',
 'lat

Now that we have selected the exchanges, we pick random criptocurrencies and verify that they are all avaliable in their platform.

In [5]:
exchanges= ['binance','ftx', 'bytetrade'] #Selected exchanges from list (can't modify because of fn and vs .py)
criptos = ['BTC/USDT','ETH/USDT','XRP/USDT'] #Criptos OBs (modify tickers here)

<font color= lightblue> <font size = 4> Order Book Levels & OHLCV for 3 exchanges and Criptos: </font>

Now we proceed to download data from exchanges for all criptocurrencies.

In [6]:
help(vs.cctx_download)

Help on function cctx_download in module visualizations:

cctx_download(lvls, cripto, exchange)
    Function that returns prices and quantities of given levels (as integer) Bids & Asks in an Orderbook
    for the specified criptocurrency ('BTC/USDT','ETH/USDT','XRP/USDT', or others) and
    from the following exchanges: binance, ftx or ascendex (as string).
    
        Parameters:
        ----------
        lvls: Levels of bids/asks in the Order Book (int).
        cripto: Criptocurrency downloadable ticker (str).
        exchanges: Criptocurrency downloadable Exchange (str).
    
        Returns:
        -------
        levels_ob_bid: Prices and Quantities of Bids (pos [0]) and Asks (pos [1]) as dataframes.



We get n=25 levels for each OB, and proceed to download combinations from criptos and exchanges from their lists [0:2] (9).

We will start with binance and BTC/USDT.

In [7]:
params= 25, criptos[0], exchanges[0]
params

(25, 'BTC/USDT', 'binance')

+ <font color= lightgray> <font size = 2> *Bids*: </font>

In [8]:
ob_lvls=vs.cctx_download(params[0],params[1], params[2])
ob_lvls[0].head()

,price,quantity
Bid_Lvl,,
0,16108.13,0.01862
1,16108.08,0.39692
2,16107.98,0.08057
3,16106.44,0.00100
4,16106.43,0.10849


+ <font color= lightgray> <font size = 2> *Asks*: </font>

In [9]:
ob_lvls[1].head()

,price,quantity
Ask_Lvl,,
0,16110.40,0.00588
1,16110.41,0.33933
2,16110.42,0.00118
3,16111.08,0.00247
4,16111.09,0.00703


In [10]:
ob_lvls[2]

,exchange,timestamp,open,high,low,close,volume
OHLCV,,,,,,,
0,binance,2022-11-09T21:06:49.125Z,16544.97,16585.43,16499.99,16523.76,915.70192


We plot the results for the first combination of ticker and exchanges with the visualization *OBLvls*.

In [11]:
help(vs.OBLvls_hist)

Help on function OBLvls_hist in module visualizations:

OBLvls_hist(lvls, cripto, exchange)
    Function that plots an horizontal histogram in plotly for CriptoCurrencies OB/
    
        Parameters
        ----------
        lvls: Levels of bids/asks in the Order Book (int).
        cripto: Criptocurrency downloadable ticker (str).
        exchanges: Criptocurrency downloadable Exchange (str).
    
        x: Quantity (col) of the given cripto (str) for given lvls (int) in Order Book.
        y: Prices (col) of the given cripto (str) for given lvls (int) in Order Book.
    
        Returns
        -------
        Histogram of Order Book prices and quantities for n given lvls.



In [12]:
vs.OBLvls_hist(params[0],params[1], params[2])

We will proceed for binance and ETH/USDT.

In [13]:
params= 25, criptos[1], exchanges[0]
params

(25, 'ETH/USDT', 'binance')

In [14]:
ob_lvls=vs.cctx_download(params[0],params[1], params[2])
ob_lvls[0].head()

,price,quantity
Bid_Lvl,,
0,1153.45,1.1762
1,1153.43,0.1596
2,1153.29,3.2112
3,1153.23,0.1000
4,1153.20,0.2484


In [15]:
ob_lvls[1].head()

,price,quantity
Ask_Lvl,,
0,1153.46,2.1393
1,1153.76,0.0344
2,1153.77,0.0172
3,1154.23,0.0596
4,1154.29,0.8700


In [16]:
ob_lvls[2]

,exchange,timestamp,open,high,low,close,volume
OHLCV,,,,,,,
0,binance,2022-11-09T21:07:09.419Z,1170.48,1170.72,1157.18,1159.91,4682.6933


In [17]:
vs.OBLvls_hist(params[0],params[1], params[2])

We continue with the exchange binance and XRP/USDT.

In [18]:
params= 25, criptos[2], exchanges[0]
params

(25, 'XRP/USDT', 'binance')

In [19]:
ob_lvls=vs.cctx_download(params[0],params[1], params[2])
ob_lvls[0].head()

,price,quantity
Bid_Lvl,,
0,0.3447,749.0
1,0.3446,450.0
2,0.3445,28397.0
3,0.3444,519385.0
4,0.3443,23088.0


In [20]:
ob_lvls[1].head()

,price,quantity
Ask_Lvl,,
0,0.3448,309.0
1,0.3449,1391.0
2,0.3450,7538.0
3,0.3451,21923.0
4,0.3452,13678.0


In [21]:
ob_lvls[2]

,exchange,timestamp,open,high,low,close,volume
OHLCV,,,,,,,
0,binance,2022-11-09T21:07:27.065Z,0.3522,0.3524,0.3486,0.3495,1324857.0


In [22]:
vs.OBLvls_hist(params[0],params[1], params[2])

We could retrieve criptocurrencies with any of the 3 exchanges declared in our list *exchanges* but to avoid being repetitive we will make one more random example and we will conclude with a dataframe. For section section **2.** *Microstructure Visualization* a dataframe with different cols and plots that allow us to compare differences between exchanges will be created.

In [23]:
params= 25, criptos[rand.randint(0,2)], exchanges[rand.randint(0,2)]
params

(25, 'BTC/USDT', 'bytetrade')

In [24]:
ob_lvls=vs.cctx_download(params[0],params[1], params[2])

In [25]:
vs.OBLvls_hist(params[0],params[1], params[2])

In [26]:
v_params0= 25, criptos[0], exchanges[0]
v_params1= 25, criptos[0], exchanges[1]
v_params2= 25, criptos[0], exchanges[2]
v_params0, v_params1, v_params2

((25, 'BTC/USDT', 'binance'),
 (25, 'BTC/USDT', 'ftx'),
 (25, 'BTC/USDT', 'bytetrade'))

<strong> 
<font color= lightgreen> <font size = 3> Progress check </font>

In [28]:
help(vs.verif_ex1)

Help on function verif_ex1 in module visualizations:

verif_ex1(lvls, cripto, exchange, n)
    Function that returns Verif 1 for i levels (as integer) n times 
    from specified criptocurrency ('BTC/USDT','ETH/USDT','XRP/USDT', or others) and the 
    following exchanges: binance, ftx or bytetrade (as string).
    
    Parameters:
    ----------
    lvls: Levels of bids/asks in the Order Book (int).
    cripto: Criptocurrency downloadable ticker (str).
    exchanges: Criptocurrency downloadable Exchange (str).
    n: Data retrieval per exchange (int).
    
    Returns:
    -------
    df1 = Section 1 Verification.
    df2 = OHLCV.



In [29]:
n=20
verif_0_B = vs.verif_ex1(v_params0[0],v_params0[1], v_params0[2], n)
verif_1_B = vs.verif_ex1(v_params1[0],v_params1[1], v_params1[2], n)
verif_2_B = vs.verif_ex1(v_params2[0],v_params2[1], v_params2[2], n)

Some differences between criptocurrencies within exchanges can be appreciated. Now we can realize which exchange from the sample can provide us with the lowest spread and best liquidity which is binance in this case. 

Clearly, prices are the most important factor for investments and that's why Mid-Price is going to be analyzed in the plots of section *2° (Microstructure Visualization)*. 

<font color= lightblue> <font size = 2> Verification BTC </font>

In [30]:
verif_df1 = fn.concatenate(verif_0_B[0], verif_1_B[0], verif_2_B[0], n)
verif_df1

,exchange,timestamp,ask,bid,ask_volume,bid_volume,spread,close_price
0,binance,2022-11-09T21:08:28.778,16103.93,16101.24,3.52827,2.806260,2.69,16102.43
1,binance,2022-11-09T21:08:31.328,16108.42,16104.63,1.35223,2.621970,3.79,16104.40
2,binance,2022-11-09T21:08:33.862,16097.19,16094.05,3.00281,1.776850,3.14,16094.18
3,binance,2022-11-09T21:08:36.546,16091.50,16086.99,2.18451,1.328340,4.51,16086.97
4,binance,2022-11-09T21:08:39.638,16073.72,16070.57,2.42195,4.752510,3.15,16073.97
5,binance,2022-11-09T21:08:42.777,16092.75,16089.71,2.06922,4.319100,3.04,16086.04
6,binance,2022-11-09T21:08:45.301,16101.29,16098.92,2.93113,1.810290,2.37,16100.33
7,binance,2022-11-09T21:08:47.937,16090.43,16088.77,3.76930,4.553870,1.66,16091.13
8,binance,2022-11-09T21:08:51.211,16096.38,16093.85,1.27687,4.310430,2.53,16092.15
9,binance,2022-11-09T21:08:53.802,16105.84,16103.03,3.49916,2.949850,2.81,16102.97


<font color= lightblue> <font size = 2> OHLCV BTC </font>

For each timeframe a candle is formed and it illustrates how much Volatility every timestamp shows within exchanges and this is important to know in algorithmic trading. Not to mention the fact that stock exchanges do not provide candle data in ms for many reasons.

In [31]:
verif_df1_OH = fn.concatenate(verif_0_B[1], verif_1_B[1], verif_2_B[1], n)
verif_df1_OH

,exchange,timestamp,open,high,low,close,volume
0,binance,2022-11-09T21:08:28.778,16091.68,16118.31,16070.02,16102.43,185.54006
1,binance,2022-11-09T21:08:31.328,16091.68,16118.31,16070.02,16104.40,196.91117
2,binance,2022-11-09T21:08:33.862,16091.68,16118.31,16070.02,16094.18,216.29950
3,binance,2022-11-09T21:08:36.546,16091.68,16118.31,16070.02,16086.97,235.45948
4,binance,2022-11-09T21:08:39.638,16091.68,16118.31,16070.00,16073.97,271.87917
5,binance,2022-11-09T21:08:42.777,16091.68,16118.31,16070.00,16086.04,288.64869
6,binance,2022-11-09T21:08:45.301,16091.68,16118.31,16070.00,16100.33,309.87142
7,binance,2022-11-09T21:08:47.937,16091.68,16118.31,16070.00,16091.13,325.50124
8,binance,2022-11-09T21:08:51.211,16091.68,16118.31,16070.00,16092.15,333.22215
9,binance,2022-11-09T21:08:53.802,16091.68,16118.31,16070.00,16102.97,351.33733


<font color= lightblue> <font size = 2> Verification ETH </font>

In [33]:
v_params0= 25, criptos[1], exchanges[0]
v_params1= 25, criptos[1], exchanges[1]
v_params2= 25, criptos[1], exchanges[2]
v_params0, v_params1, v_params2

((25, 'ETH/USDT', 'binance'),
 (25, 'ETH/USDT', 'ftx'),
 (25, 'ETH/USDT', 'bytetrade'))

In [34]:
n=20
verif_0_E = vs.verif_ex1(v_params0[0],v_params0[1], v_params0[2], n)
verif_1_E = vs.verif_ex1(v_params1[0],v_params1[1], v_params1[2], n)
verif_2_E = vs.verif_ex1(v_params2[0],v_params2[1], v_params2[2], n)

In [35]:
verif_df2 = fn.concatenate(verif_0_E[0], verif_1_E[0], verif_2_E[0], n)
verif_df2

,exchange,timestamp,ask,bid,ask_volume,bid_volume,spread,close_price
0,binance,2022-11-09T21:14:00.755,1153.76,1153.75,33.8743,59.8397,0.01,1153.76
1,binance,2022-11-09T21:14:03.422,1155.40,1155.39,46.4509,32.4389,0.01,1155.10
2,binance,2022-11-09T21:14:06.016,1153.90,1153.60,37.2692,135.7212,0.30,1153.60
3,binance,2022-11-09T21:14:08.639,1154.36,1154.35,43.2427,31.7348,0.01,1154.22
4,binance,2022-11-09T21:14:11.166,1154.32,1154.15,114.6540,65.0012,0.17,1154.29
5,binance,2022-11-09T21:14:13.687,1153.44,1153.43,106.1127,64.5498,0.01,1153.58
6,binance,2022-11-09T21:14:16.324,1153.58,1153.57,29.5727,54.9679,0.01,1153.58
7,binance,2022-11-09T21:14:19.363,1153.49,1153.48,61.0482,58.8651,0.01,1153.49
8,binance,2022-11-09T21:14:21.979,1152.11,1151.94,55.6483,26.7995,0.17,1151.89
9,binance,2022-11-09T21:14:24.535,1151.97,1151.72,69.4939,38.2516,0.25,1151.89


<font color= lightblue> <font size = 2> OHLCV ETH </font>

In [36]:
verif_df2_OH = fn.concatenate(verif_0_E[1], verif_1_E[1], verif_2_E[1], n)
verif_df2_OH

,exchange,timestamp,open,high,low,close,volume
0,binance,2022-11-09T21:14:00.755,1148.57,1157.91,1147.49,1153.76,1841.2213
1,binance,2022-11-09T21:14:03.422,1155.11,1155.75,1154.37,1155.10,9.5498
2,binance,2022-11-09T21:14:06.016,1155.11,1155.75,1151.61,1153.60,213.4811
3,binance,2022-11-09T21:14:08.639,1155.11,1155.75,1151.61,1154.22,226.8846
4,binance,2022-11-09T21:14:11.166,1155.11,1155.75,1151.61,1154.29,235.7805
5,binance,2022-11-09T21:14:13.687,1155.11,1155.75,1151.61,1153.58,251.8282
6,binance,2022-11-09T21:14:16.324,1155.11,1155.75,1151.61,1153.58,325.9377
7,binance,2022-11-09T21:14:19.363,1155.11,1155.75,1151.61,1153.49,343.8879
8,binance,2022-11-09T21:14:21.979,1155.11,1155.75,1151.61,1151.89,391.6930
9,binance,2022-11-09T21:14:24.535,1155.11,1155.75,1151.50,1151.89,429.3198


<font color= lightblue> <font size = 2> Verification XRP </font>

In [37]:
v_params0= 25, criptos[2], exchanges[0]
v_params1= 25, criptos[2], exchanges[1]
v_params2= 25, criptos[2], exchanges[2]
v_params0, v_params1, v_params2

((25, 'XRP/USDT', 'binance'),
 (25, 'XRP/USDT', 'ftx'),
 (25, 'XRP/USDT', 'bytetrade'))

In [41]:
n=20
verif_0_X = vs.verif_ex1(v_params0[0],v_params0[1], v_params0[2], n)
verif_1_X = vs.verif_ex1(v_params1[0],v_params1[1], v_params1[2], n)
verif_2_X = vs.verif_ex1(v_params2[0],v_params2[1], v_params2[2], n)

In [42]:
verif_df3 = fn.concatenate(verif_0_X[0], verif_1_X[0], verif_2_X[0], n)
verif_df3

,exchange,timestamp,ask,bid,ask_volume,bid_volume,spread,close_price
0,binance,2022-11-09T21:20:10.820,0.338400,0.337700,398347.000000,1.641913e+06,0.000700,0.337500
1,binance,2022-11-09T21:20:13.180,0.338000,0.337900,388503.000000,1.820410e+06,0.000100,0.337900
2,binance,2022-11-09T21:20:16.793,0.338000,0.337700,429096.000000,1.643038e+06,0.000300,0.337800
3,binance,2022-11-09T21:20:19.707,0.338000,0.337800,377075.000000,1.730288e+06,0.000200,0.337900
4,binance,2022-11-09T21:20:22.205,0.337800,0.337700,402211.000000,1.820201e+06,0.000100,0.337700
5,binance,2022-11-09T21:20:24.685,0.337400,0.337200,363172.000000,3.858636e+06,0.000200,0.337100
6,binance,2022-11-09T21:20:27.202,0.338000,0.337800,395261.000000,1.532947e+06,0.000200,0.337900
7,binance,2022-11-09T21:20:29.697,0.337900,0.337700,382131.000000,1.510881e+06,0.000200,0.337800
8,binance,2022-11-09T21:20:32.344,0.337900,0.337800,379385.000000,1.696193e+06,0.000100,0.337800
9,binance,2022-11-09T21:20:35.015,0.338100,0.338000,408767.000000,1.612673e+06,0.000100,0.338100


<font color= lightblue> <font size = 2> OHLCV XRP </font>

In [43]:
verif_df3_OH = fn.concatenate(verif_0_X[1], verif_1_X[1], verif_2_X[1], n)
verif_df3_OH

,exchange,timestamp,open,high,low,close,volume
0,binance,2022-11-09T21:20:10.820,0.337700,0.338000,0.337100,0.337500,88493.000000
1,binance,2022-11-09T21:20:13.180,0.337700,0.338300,0.337100,0.337900,100567.000000
2,binance,2022-11-09T21:20:16.793,0.337700,0.338300,0.337100,0.337800,114224.000000
3,binance,2022-11-09T21:20:19.707,0.337700,0.338300,0.337100,0.337900,122114.000000
4,binance,2022-11-09T21:20:22.205,0.337700,0.338300,0.337100,0.337700,130254.000000
5,binance,2022-11-09T21:20:24.685,0.337700,0.338300,0.337100,0.337100,183343.000000
6,binance,2022-11-09T21:20:27.202,0.337700,0.338300,0.337100,0.337900,215625.000000
7,binance,2022-11-09T21:20:29.697,0.337700,0.338300,0.337100,0.337800,230245.000000
8,binance,2022-11-09T21:20:32.344,0.337700,0.338300,0.337100,0.337800,239147.000000
9,binance,2022-11-09T21:20:35.015,0.337700,0.338300,0.337100,0.338100,257489.000000


<font color= orange> <font size = 5> 2. Microstructure Visualization </font>

Now we are going to obtain the total_volume, mid_price and VWAP for criptocurrencies in every exchange.

In [58]:
help(vs.Micro)

Help on function Micro in module visualizations:

Micro(lvls, cripto, exchange, n, ts)
    Function that returns Microstructure OB and OHLCV for i levels (as integer) n times 
    from specified criptocurrency ('BTC/USDT','ETH/USDT','XRP/USDT', or others) and the 
    following exchanges: binance, ftx or bytetrade (as string).
    
    Parameters:
    ----------
    lvls: Levels of bids/asks in the Order Book (int).
    cripto: Criptocurrency downloadable ticker (str).
    exchanges: Criptocurrency downloadable Exchange (str).
    n: Data retrieval per exchange (int).
    ts: timesleep (s) required in between data retrieval.
    
    Returns:
    -------
    Micro[0]: Order Books Data.
    Micro[1]: OHLCV Data.



In [59]:
params0= 25, criptos[0], exchanges[0]
params1= 25, criptos[0], exchanges[1]
params2= 25, criptos[0], exchanges[2]
params0,params1,params2

((25, 'BTC/USDT', 'binance'),
 (25, 'BTC/USDT', 'ftx'),
 (25, 'BTC/USDT', 'bytetrade'))

<strong> 
<font color= lightgreen> <font size = 3> Progress check 2.1 </font>

In [61]:
n=20
MS_0 = vs.Micro(params0[0],params0[1], params0[2], n, .01)
MS_1 = vs.Micro(params1[0],params1[1], params1[2], n, .01)
MS_2 = vs.Micro(params2[0],params2[1], params2[2], n, .01)

Bid and Asks Volumes in the Microstructure determine the demand and it's a factor for price movements. This is also true for the VWAP which is a trend detector by reflecting the price adjusted by its volume.

In [62]:
MSob_df = fn.concatenate(MS_0[0], MS_1[0], MS_2[0], n)
MSob_df

,exchange,timestamp,level,ask_volume,bid_volume,total_volume,mid_price,vwap
0,binance,2022-11-09T21:36:09.888,25,6.95810,2.056230,9.014330,16145.545,17935.304575
1,binance,2022-11-09T21:36:11.431,25,7.92624,2.806930,10.733170,16142.890,17646.718415
2,binance,2022-11-09T21:36:13.083,25,3.04364,3.836780,6.880420,16137.950,18483.199779
3,binance,2022-11-09T21:36:14.509,25,2.01445,0.993180,3.007630,16133.380,21497.043227
4,binance,2022-11-09T21:36:15.872,25,2.43277,0.829020,3.261790,16123.580,21066.478808
5,binance,2022-11-09T21:36:17.590,25,3.71112,1.232510,4.943630,16125.975,19387.502721
6,binance,2022-11-09T21:36:19.240,25,2.19635,2.108630,4.304980,16131.920,19877.830593
7,binance,2022-11-09T21:36:20.746,25,2.19647,1.324380,3.520850,16125.100,20704.744947
8,binance,2022-11-09T21:36:22.140,25,1.96892,1.854620,3.823540,16120.755,20336.937094
9,binance,2022-11-09T21:36:23.567,25,1.56081,1.564200,3.125010,16117.685,21274.181894


In [63]:
MSohlcv = fn.concatenate(MS_0[1], MS_1[1], MS_2[1], n)
MSohlcv

,exchange,timestamp,open,high,low,close,volume
0,binance,2022-11-09T21:36:09.888,16137.66,16149.86,16130.00,16144.06,35.15998
1,binance,2022-11-09T21:36:11.431,16137.66,16149.86,16130.00,16145.33,42.81799
2,binance,2022-11-09T21:36:13.083,16137.66,16149.86,16130.00,16130.13,52.52006
3,binance,2022-11-09T21:36:14.509,16137.66,16150.00,16129.13,16136.16,81.45285
4,binance,2022-11-09T21:36:15.872,16137.66,16150.00,16124.13,16124.26,94.45078
5,binance,2022-11-09T21:36:17.590,16137.66,16150.00,16123.14,16126.04,100.84773
6,binance,2022-11-09T21:36:19.240,16137.66,16150.00,16120.61,16134.93,116.29978
7,binance,2022-11-09T21:36:20.746,16137.66,16150.00,16117.59,16125.42,120.68978
8,binance,2022-11-09T21:36:22.140,16137.66,16150.00,16116.49,16122.34,135.31651
9,binance,2022-11-09T21:36:23.567,16137.66,16150.00,16114.54,16118.80,138.43121


In [64]:
p = MSob_df[['exchange', 'mid_price']]
p.index =pd.to_datetime(MSob_df['timestamp'])
p.head()

,exchange,mid_price
timestamp,,
2022-11-09 21:36:09.888,binance,16145.545
2022-11-09 21:36:11.431,binance,16142.890
2022-11-09 21:36:13.083,binance,16137.950
2022-11-09 21:36:14.509,binance,16133.380
2022-11-09 21:36:15.872,binance,16123.580


In [65]:
fig = go.Figure(layout_xaxis_range=[0,10])
fig = px.line(p, facet_col="exchange", facet_col_wrap=1)
fig.update_layout(title_text="Mid-Price of " + str(params0[1]) + str(':'),
                title_font_size=15,)
fig.show()

<font color= orange> <font size = 5> 3. Microstructure Modelling: </font>

### <font color= orange> <font size = 5> 4. Bibliography: </font>

<font color= lightgray>

+ Villalobos, O, 2021. **Python Project Template**. Available at: https://github.com/OmarVillalobos/python-project. 
(Accessed: November 5, 2022). 

+ Financial Engineering, **Marketmicrostructure.pptx**. ITESO -  Microstructure and Trading Systems (ITE1731 - DMAF). 
Available at: https://docs.google.com/presentation/d/1uYyMERKKGeZ0MPqbJ3bcir9lfpzbeAH9ZMmxbYysokQ/edit?usp=sharing (Accessed: November 5, 2022). 

+ Muñoz E., J **Roll Model** *(spanish)*. ITESO - (ITE1731 - DMAF) Available at: https://github.com/EstebanMqz/MyST_LAB_4/blob/main/Roll%20.pdf (Accessed: November 5, 2022).

+ Muñoz E., J **Asset Pricing Theory** *(spanish)*. ITESO - (ITE1731 - DMAF) Available at: https://github.com/EstebanMqz/MyST_LAB_4/blob/main/APT.pdf (Accessed: November 5, 2022). 